In [2]:
import pandas as pd
import numpy as np

In [3]:
data = {
    'x1': [1, 1, 0, 0],
    'x2': [0, 0, 0, 1],
    'x3': [1, 0, 0, 0],
    'x4': [0, 0, 1, 1]
}
data = pd.DataFrame(data, dtype='float')
data

,x1,x2,x3,x4
0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0


In [88]:
y = np.array([0, 1, 0, 1])

In [85]:
class nnKohonen:

    def __init__(self, n_cohonen, n_grossberg, epochs=50):
        self.n_cohonen = n_cohonen
        self.n_grossberg = n_grossberg
        self.epochs = epochs
        self.weights_coh = None
        self.weights_gr = None

    def layer_kohonen(self, input, nu):
        input = np.asarray(input, dtype=float)
        distances = []
        for n in range(self.n_cohonen):
            distances.append(np.sqrt(np.sum(np.pow((input - self.weights_coh[n]), 2))))
        n_winner = np.argmin(distances)
        self.weights_coh[n_winner] += nu * (input - self.weights_coh[n_winner])
    
        out = np.zeros(self.n_cohonen, dtype=float)
        out[n_winner] = 1.0
        return out
    
    def layer_grossberg(self, output_koh, beta, y):
        y_one_hot_vec = np.eye(self.n_grossberg, dtype=float)[int(y)]

        delta = beta * (y_one_hot_vec[:, None] - self.weights_gr) * output_koh[None, :]
        self.weights_gr += delta

    def fit(self, data, y):
        nu = 0.6 # коэффициент обучения для сети кохонена
        beta = 0.1 # коэффициент обучения для сети гройсберга

        n_features = data.shape[1] # кол-во обучаемых векторов
        self.weights_coh = np.ones((self.n_cohonen, n_features)) / np.sqrt(self.n_cohonen)

        self.weights_gr = np.random.rand(self.n_grossberg, self.n_cohonen)

        for _ in range(self.epochs):    
            for i, col in enumerate(data.columns):
                train_x = data[col].values        
                train_y = y[i]

                outputs_kohonen = self.layer_kohonen(train_x, nu)
                self.layer_grossberg(outputs_kohonen, beta, train_y)

            if nu >= 0.05:
                nu -= 0.01
            if beta >= 0.0001:  
                beta -= 0.005
        

    def predict(self, data):
        clusters = []
        for x in data:
            distances = []
            for n in range(self.n_cohonen):
                distances.append(np.sqrt(np.sum(np.pow((data[x] - self.weights_coh[n]), 2))))
            n_winner = np.argmin(distances)
            clusters.append(n_winner)
        clusters = np.array(clusters)
        output = self.weights_gr[:, clusters].T
        predicted_classes = np.argmax(output, axis=1)
        return predicted_classes


In [86]:
nn = nnKohonen(n_cohonen=2, n_grossberg=2, epochs=100)

In [89]:
nn.fit(data, y)

In [90]:
np.round(nn.weights_coh, 2)

array([[1.  , 0.49, 0.  , 0.  ],
       [0.  , 0.  , 0.51, 1.  ]])

In [91]:
np.round(nn.weights_gr, 2)

array([[0.92, 0.01],
       [0.05, 0.93]])

In [92]:
nn.predict(data)

array([0, 1, 0, 1])